In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
import time
from classes.dataset import Dataset
from libs.location import *

2023-09-27 07:01:49.221682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 07:01:49.952457: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from tensorflow.keras import Sequential,Model
from tensorflow.keras import models,layers,optimizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16,DenseNet121
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
X = np.load(f'{DATASET_LOCATION_PATH}/X.npy')
y = np.load(f'{DATASET_LOCATION_PATH}/y.npy')

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [6]:
ds = Dataset()

##Initialising Dataset


In [8]:
model = ds.create_model()

##Creating Custom Model


AttributeError: 'Dataset' object has no attribute 'mode'

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 111, 111, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 109, 109, 64)      36928     
                                                        

In [ ]:
model.load_weights(f'{MODEL_CHECKPOINT_LOCATION_PATH}/model.19-0.44.h5')

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0)
score

[0.4400370717048645, 0.7860221266746521]

In [ ]:
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

167/167 [==============================] - 58s 336ms/step
predictions shape: (5337, 2)


In [ ]:
X_test.shape

(5337, 224, 224, 3)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
y_test_unc = [np.argmax(i) for i in y_test]
y_train_unc = [np.argmax(i) for i in y_train]
y_pred_unc = [np.argmax(i) for i in predictions]

In [ ]:
accuracy_score(y_test_unc,y_pred_unc)

0.7860221097995128

In [ ]:
print(classification_report(y_test_unc,y_pred_unc))

              precision    recall  f1-score   support

           0       0.79      0.98      0.88      4108
           1       0.65      0.15      0.25      1229

    accuracy                           0.79      5337
   macro avg       0.72      0.56      0.56      5337
weighted avg       0.76      0.79      0.73      5337



In [ ]:
CHEXNET_WEIGHTS_FILE = f'{MODEL_CHECKPOINT_LOCATION_PATH}/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'

In [ ]:
pre_model = DenseNet121(weights=None,
                                include_top=False,
                                input_shape=(224,224,3)
                               )

In [ ]:
out = Dense(14, activation='sigmoid')(pre_model.output)
chexnet_model = Model(inputs=pre_model.input, outputs=out) 

In [ ]:
chexnet_model.load_weights(filepath=f'{MODEL_CHECKPOINT_LOCATION_PATH}/brucechou1983_CheXNet_Keras_0.3.0_weights.h5')

In [ ]:
for layer in chexnet_model.layers[:-2]:
    layer.trainable = False

In [ ]:
x = chexnet_model.layers[-2].output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.1)(x)
output = Dense(2, activation='softmax')(x)

In [ ]:
chexnet_model_custom = Model(inputs=chexnet_model.input,outputs=output)

In [ ]:
chexnet_model_custom.compile(optimizer="Adam",loss='binary_crossentropy',metrics='accuracy')

In [ ]:
from datetime import datetime

In [ ]:
datetime.today().strftime('%Y-%m-%d')

'2023-09-25'

In [ ]:
def get_callbacks() -> None:
        stopping_callback = EarlyStopping(monitor="loss", patience=3)
        checkpoint_callback = ModelCheckpoint(
            filepath=f"{MODEL_CHECKPOINT_LOCATION_PATH}/model_chexnet_{datetime.today().strftime('%Y-%m-%d')}.E{{epoch:02d}}-VL{{val_loss:.2f}}-A{{accuracy:.2f}}.h5",
            save_best_only=True,
            save_weights_only=True,
            verbose=1,
        )
        return [stopping_callback, checkpoint_callback]

In [ ]:
# start = time.time()
# history = model.fit(
#             x=X_train,
#             y=y_train,
#             validation_data=(X_test, y_test),
#             batch_size=32,
#             epochs=10,
#             verbose=1,
#             callbacks=get_callbacks(),
#         )
# end = time.time()

In [ ]:
y_temp = [np.argmax(i) for i in y_train]

In [ ]:
from collections import Counter

element_counts = Counter(y_temp)
print(element_counts)

Counter({0: 16564, 1: 4783})


In [ ]:
model = ds.create_chexnet_model()

In [ ]:
model.load_weights(f'{MODEL_CHECKPOINT_LOCATION_PATH}/model_chexnet.15-0.48.h5')

In [ ]:
score = model.evaluate(X_test, y_test, verbose = 0)
score

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.